In [9]:
from chembl_webresource_client.new_client import new_client
import pandas as pd 
from chembl_webresource_client.settings import Settings 

Settings.Instance().CACHING = False 
Settings.Instance().TIMEOUT = 1000

activity = new_client.activity 
mechanism = new_client.mechanism 
molecule = new_client.molecule
target = new_client.target

target_real = 'CHEMBL287'


# to find the officisl binder of the sugma- r1


mec = mechanism.filter(target_chembl_id = target_real).only('action_type', 'molecule_chembl_id', 'parent_molecule_chembl_id', 'molecule_pref_name')
mec_df = pd.DataFrame(mec)
print(mec_df)  

mec_df.to_csv('mechanism_filter.csv', index = False )


#to find al other binders like the modulators

activities = activity.filter(target_chembl_id= [target_real]
                             , pchembl_value__gte = 5
                             , assay_type = 'B')\
                             .only('pchembl_value', 'molecule_chembl_id', 'parent_molecule_chembl_id', 'molecule_pref_name', 'parent_molecule_pref_name' )
df = pd.DataFrame(activities)
print(df)


df.to_csv('target_activity.csv', index = False )


molecule_df = pd.read_csv('target_activity.csv')

molecules = molecule_df['molecule_chembl_id'].tolist()

mec = mechanism.filter(molecule_chembl_id__in = molecules).only('action_type', 'max_phase', 'molecule_chembl_id', 'parent_molecule_chembl_id')
mec_df = pd.DataFrame(mec)
print(mec_df)

mec_df.to_csv('molecule_mechanism_filter.csv', index = False) 

activity_df = pd.read_csv('target_activity.csv')
mechanism_df = pd.read_csv('molecule_mechanism_filter.csv')

merged_table = pd.merge(activity_df, mechanism_df , on= 'molecule_chembl_id', how= 'inner')

print(merged_table.head())

dff= merged_table.to_csv('the_two_table.csv', index = False)

  action_type molecule_chembl_id parent_molecule_chembl_id
0     AGONIST      CHEMBL1201461               CHEMBL52440
1   MODULATOR      CHEMBL3989509              CHEMBL100116
2   MODULATOR      CHEMBL3989510              CHEMBL100116
3     AGONIST      CHEMBL1256818               CHEMBL52440
4     AGONIST      CHEMBL1256696               CHEMBL73234
5  ANTAGONIST      CHEMBL2106217               CHEMBL87493
     molecule_chembl_id molecule_pref_name parent_molecule_chembl_id  \
0           CHEMBL38904               None               CHEMBL38904   
1           CHEMBL67010               None               CHEMBL67010   
2          CHEMBL542638               None             CHEMBL1191528   
3           CHEMBL46169               None               CHEMBL46169   
4           CHEMBL45962               None               CHEMBL45962   
...                 ...                ...                       ...   
4242      CHEMBL5781923               None             CHEMBL5781923   
4243       

In [11]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.settings import Settings

Settings.Instance().CACHING = False
Settings.Instance().TIMEOUT = 120

activity = new_client.activity
mechanism = new_client.mechanism

target_real = 'CHEMBL287' # Sigma-1

# --- 1. GET THE 4,400 BINDERS ---
print("1. Fetching all molecules with pChEMBL >= 5...")
activities = activity.filter(
    target_chembl_id=target_real, 
    pchembl_value__gte=5, 
    assay_type='B'
).only(['pchembl_value', 'molecule_chembl_id', 'molecule_pref_name'])

# Put it in a DataFrame and drop duplicates immediately
activity_df = pd.DataFrame(list(activities)).drop_duplicates(subset=['molecule_chembl_id'])
print(f"Total unique binders found: {len(activity_df)}")

# --- 2. BATCH FETCH THE MECHANISMS (The Fix!) ---
# Extract the clean list of IDs
molecules = activity_df['molecule_chembl_id'].tolist()
all_mechanisms = []

print("\n2. Fetching mechanisms in safe batches to prevent URL truncation...")
# We slice the massive list into tiny chunks of 100
for i in range(0, len(molecules), 100):
    batch = molecules[i : i + 100]
    
    # Send only 100 IDs at a time
    mec = mechanism.filter(molecule_chembl_id__in=batch)\
                   .only(['action_type', 'molecule_chembl_id'])
                   
    all_mechanisms.extend(list(mec))
    print(f"  Processed {min(i + 100, len(molecules))} / {len(molecules)}...")

mechanism_df = pd.DataFrame(all_mechanisms)

# --- 3. MERGE THE TABLES ---
print("\n3. Merging the data...")
if not mechanism_df.empty:
    # Use an INNER merge to only keep the molecules that actually have an official mechanism
    merged_table = pd.merge(activity_df, mechanism_df, on='molecule_chembl_id', how='inner')
    
    # Clean up and display
    print(f"\n✅ SUCCESS! Out of {len(activity_df)} raw binders, we found {len(merged_table)} official clinical mechanisms.")
    print(merged_table.head(15))
    
    merged_table.to_csv('Master_Combined_Table.csv', index=False)
    print("\nSaved securely to 'Master_Combined_Table.csv'")
else:
    print("No mechanisms were found.")

1. Fetching all molecules with pChEMBL >= 5...
Total unique binders found: 3140

2. Fetching mechanisms in safe batches to prevent URL truncation...
  Processed 100 / 3140...
  Processed 200 / 3140...
  Processed 300 / 3140...
  Processed 400 / 3140...
  Processed 500 / 3140...
  Processed 600 / 3140...
  Processed 700 / 3140...
  Processed 800 / 3140...
  Processed 900 / 3140...
  Processed 1000 / 3140...
  Processed 1100 / 3140...
  Processed 1200 / 3140...
  Processed 1300 / 3140...
  Processed 1400 / 3140...
  Processed 1500 / 3140...
  Processed 1600 / 3140...
  Processed 1700 / 3140...
  Processed 1800 / 3140...
  Processed 1900 / 3140...
  Processed 2000 / 3140...
  Processed 2100 / 3140...
  Processed 2200 / 3140...
  Processed 2300 / 3140...
  Processed 2400 / 3140...
  Processed 2500 / 3140...
  Processed 2600 / 3140...
  Processed 2700 / 3140...
  Processed 2800 / 3140...
  Processed 2900 / 3140...
  Processed 3000 / 3140...
  Processed 3100 / 3140...
  Processed 3140 / 3140